In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [3]:
msg_paths = sorted(glob.glob("../data/results/*.csv"))

df = pd.read_csv(msg_paths[0])
df

,timestamp,TimeUS,Roll,Pitch,Yaw,Alt,Lat,Lng,Q1,Q2,Q3,Q4,label
0,2024-11-12 14:02:01,287589565,0.88,0.73,78.79,70.110001,35.928239,128.871155,0.772733,0.001907,0.009879,0.634652,0
1,2024-11-12 14:02:01,287688106,0.88,0.73,78.79,70.199997,35.928239,128.871155,0.772718,0.001856,0.009852,0.634671,0
2,2024-11-12 14:02:02,287788037,0.87,0.73,78.79,70.099998,35.928239,128.871155,0.772720,0.001804,0.009811,0.634669,0
3,2024-11-12 14:02:02,287888029,0.87,0.73,78.79,70.290001,35.928239,128.871155,0.772763,0.001860,0.009781,0.634617,0
4,2024-11-12 14:02:02,287988089,0.88,0.72,78.78,70.139999,35.928239,128.871155,0.772776,0.001957,0.009758,0.634601,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125966,2024-11-15 14:50:43,1370091355,0.34,-0.06,253.20,72.439995,35.928328,128.871678,0.596299,0.001361,-0.002779,-0.802756,1
125967,2024-11-15 14:50:43,1370191390,0.35,-0.06,253.23,72.610001,35.928328,128.871678,0.596457,0.001387,-0.002793,-0.802639,1
125968,2024-11-15 14:50:43,1370291411,0.35,-0.05,253.24,72.639999,35.928328,128.871678,0.596554,0.001480,-0.002749,-0.802567,1
125969,2024-11-15 14:50:43,1370391419,0.35,-0.05,253.26,72.659996,35.928328,128.871678,0.596663,0.001463,-0.002817,-0.802486,1


In [19]:
n_df

,Roll,Pitch,Yaw,Alt,Lat,Lng,Q1,Q2,Q3,Q4,label
0,0.88,0.73,78.79,70.110001,35.928239,128.871155,0.772733,0.001907,0.009879,0.634652,0
1,0.88,0.73,78.79,70.199997,35.928239,128.871155,0.772718,0.001856,0.009852,0.634671,0
2,0.87,0.73,78.79,70.099998,35.928239,128.871155,0.772720,0.001804,0.009811,0.634669,0
3,0.87,0.73,78.79,70.290001,35.928239,128.871155,0.772763,0.001860,0.009781,0.634617,0
4,0.88,0.72,78.78,70.139999,35.928239,128.871155,0.772776,0.001957,0.009758,0.634601,0
...,...,...,...,...,...,...,...,...,...,...,...
125966,0.34,-0.06,253.20,72.439995,35.928328,128.871678,0.596299,0.001361,-0.002779,-0.802756,1
125967,0.35,-0.06,253.23,72.610001,35.928328,128.871678,0.596457,0.001387,-0.002793,-0.802639,1
125968,0.35,-0.05,253.24,72.639999,35.928328,128.871678,0.596554,0.001480,-0.002749,-0.802567,1
125969,0.35,-0.05,253.26,72.659996,35.928328,128.871678,0.596663,0.001463,-0.002817,-0.802486,1


In [ ]:
from pycaret.classification import *

df = pd.read_csv(msg_paths[1])
n_df = df.drop(["timestamp", "TimeUS"], axis=1)

clf1 = setup(data=n_df, target='label', session_id=42, verbose=False)

tree_models = ['rf', 'et', 'dt', 'ada', 'gbc']
best_model = compare_models(include=tree_models)

sample_df = df.sample(n=3000, random_state=42)  # 3000~5000개로 충분

# 4. 모델 해석 (SHAP 기반)
# interpret_model(best_model)

# 5. 평가 지표 확인
# evaluate_model(best_model)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9993,1.0000,0.9989,0.9997,0.9993,0.9986,0.9986,0.4440
rf,Random Forest Classifier,0.9987,1.0000,0.9978,0.9996,0.9987,0.9973,0.9973,0.7120
dt,Decision Tree Classifier,0.9977,0.9977,0.9980,0.9974,0.9977,0.9954,0.9954,0.0400
gbc,Gradient Boosting Classifier,0.9931,0.9996,0.9921,0.9942,0.9932,0.9862,0.9862,0.7460
ada,Ada Boost Classifier,0.9830,0.9988,0.9856,0.9808,0.9832,0.9659,0.9660,0.1850


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

/home/seobin1027/miniconda3/envs/drone/lib/python3.11/site-packages/pycaret/internal/pycaret_experiment/supervised_experiment.py:339: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(highlight_cols, subset=["TT (Sec)"])


In [ ]:
from pycaret.classification import *
import shap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


for i in range(len(msg_paths)):
    msg_name = os.path.basename(msg_paths[i]).split('_')[0]
    print(msg_name)

    df = pd.read_csv(msg_paths[i])
    n_df = df.drop(["timestamp", "TimeUS"], axis=1)

    clf1 = setup(data=n_df, target='label', session_id=42, verbose=False)

    tree_models = ['rf', 'et', 'dt', 'ada', 'gbc']
    best_model = compare_models(include=tree_models)

    normal_sample = n_df[n_df['label'] == 0].sample(n=2500, random_state=42)
    anomaly_sample = n_df[n_df['label'] == 1].sample(n=2500, random_state=42)
    sample_df = pd.concat([normal_sample, anomaly_sample])

    X = sample_df.drop(columns=['label'])
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X)

    # 평균 절댓값 기준 feature 중요도 계산
    shap_importance = pd.DataFrame({
        'feature': X.columns,
        'mean_abs_shap': np.abs(shap_values[:, :, 1]).mean(axis=0)
    })

    # 중요도 높은 순으로 정렬
    shap_importance = shap_importance.sort_values(by='mean_abs_shap', ascending=False)
    print(f"{msg_name} SHAP 중요도 TOP 10")
    print(shap_importance.head(10))
    



# 4. 모델 해석 (SHAP 기반)
# interpret_model(best_model)

# 5. 평가 지표 확인
# evaluate_model(best_model)


AHR2


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7520
et,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1970
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3610
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.7380
dt,Decision Tree Classifier,0.9999,0.9999,0.9999,0.9999,0.9999,0.9998,0.9998,0.0430


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

/home/seobin1027/miniconda3/envs/drone/lib/python3.11/site-packages/pycaret/internal/pycaret_experiment/supervised_experiment.py:339: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(highlight_cols, subset=["TT (Sec)"])


AHR2 SHAP 중요도 TOP 10
  feature  mean_abs_shap
4     Lat       0.146429
3     Alt       0.135043
0    Roll       0.069048
5     Lng       0.066015
2     Yaw       0.059459
6      Q1       0.034730
9      Q4       0.024857
7      Q2       0.017314
8      Q3       0.008892
1   Pitch       0.001470


In [40]:
from pycaret.classification import *
import shap
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 전체 결과를 저장할 리스트
all_shap_results = []

for i in range(len(msg_paths)):
    msg_name = os.path.basename(msg_paths[i]).split('_')[0]
    print(f"{msg_name} Tree모델 학습 및 Best Model SHAP 분석")

    try:
        # 데이터 로딩 및 정제
        df = pd.read_csv(msg_paths[i])
        n_df = df.drop(["timestamp", "TimeUS"], axis=1)

        # PyCaret 학습
        clf1 = setup(data=n_df, target='label', session_id=42, verbose=False)
        tree_models = ['rf', 'et', 'dt', 'ada', 'gbc']
        best_model = compare_models(include=tree_models)

        # 클래스 균형 샘플링
        normal_sample = n_df[n_df['label'] == 0].sample(n=1500, random_state=42)
        anomaly_sample = n_df[n_df['label'] == 1].sample(n=1500, random_state=42)
        sample_df = pd.concat([normal_sample, anomaly_sample])
        X = sample_df.drop(columns=['label'])

        # SHAP 계산
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X)

        if isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
            shap_array = shap_values[:, :, 1]
        elif isinstance(shap_values, list):
            shap_array = np.array(shap_values[1])
        else:
            shap_array = np.array(shap_values)

        # 중요도 계산 및 필드 이름 추가
        shap_importance = pd.DataFrame({
            'feature': X.columns,
            'mean_abs_shap': np.abs(shap_array).mean(axis=0)
        })
        shap_importance['field'] = msg_name
        shap_importance = shap_importance.sort_values(by='mean_abs_shap', ascending=False)

        # 누적 저장
        all_shap_results.append(shap_importance)

    except Exception as e:
        print(f"XXXXX {msg_name} 분석 실패: {e} XXXXX")
        continue

# 전체 통합 테이블 생성 및 저장
all_shap_df = pd.concat(all_shap_results, ignore_index=True)
os.makedirs("shap_results", exist_ok=True)
all_shap_df.to_csv("shap_results/all_fields_shap_importance.csv", index=False)
print("\n 모든 필드 SHAP 중요도 통합 저장 완료: shap_results/all_fields_shap_importance.csv")



AHR2 Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7040
et,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3640
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3540
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.0080
dt,Decision Tree Classifier,0.9999,0.9999,0.9999,0.9999,0.9999,0.9998,0.9998,0.0440


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

ATT Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9993,1.0000,0.9989,0.9997,0.9993,0.9986,0.9986,0.2800
rf,Random Forest Classifier,0.9987,1.0000,0.9978,0.9996,0.9987,0.9973,0.9973,0.4710
dt,Decision Tree Classifier,0.9977,0.9977,0.9980,0.9974,0.9977,0.9954,0.9954,0.0370
gbc,Gradient Boosting Classifier,0.9931,0.9996,0.9921,0.9942,0.9932,0.9862,0.9862,0.7340
ada,Ada Boost Classifier,0.9830,0.9988,0.9856,0.9808,0.9832,0.9659,0.9660,0.1770


...

Processing:   0%|          | 0/25 [00:00<?, ?it/s]

PM Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9615,0.9875,0.9686,0.9567,0.9622,0.9230,0.9241,0.0110
gbc,Gradient Boosting Classifier,0.9547,0.9886,0.9685,0.9441,0.9557,0.9094,0.9107,0.0160
et,Extra Trees Classifier,0.9524,0.9895,0.9462,0.9601,0.9524,0.9049,0.9062,0.0270
rf,Random Forest Classifier,0.9513,0.9901,0.9552,0.9500,0.9518,0.9026,0.9042,0.0360
dt,Decision Tree Classifier,0.9400,0.9401,0.9305,0.9509,0.9396,0.8801,0.8819,0.0050


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

XXXXX PM 분석 실패: Cannot take a larger sample than population when 'replace=False' XXXXX

 모든 필드 SHAP 중요도 통합 저장 완료: shap_results/all_fields_shap_importance.csv


In [29]:
X

,Roll,Pitch,Yaw,Alt,Lat,Lng,Q1,Q2,Q3,Q4
59381,-0.68,0.78,89.60,57.809998,35.928297,128.871330,0.709436,-0.009076,0.000667,0.704711
59626,-1.13,1.09,90.04,57.439999,35.928297,128.871330,0.706666,-0.013781,-0.000258,0.707413
31729,-0.97,1.04,91.41,59.309998,35.928285,128.871342,0.698190,-0.012420,0.000269,0.715805
18855,-1.09,-1.66,81.25,61.329998,35.928309,128.871338,0.758930,0.002224,-0.017214,0.650941
42350,-0.67,0.50,85.13,53.919998,35.928271,128.871269,0.736438,-0.007312,-0.000683,0.676465
...,...,...,...,...,...,...,...,...,...,...
79012,1.04,0.09,84.09,73.409996,35.928247,128.871299,0.742566,0.006211,0.006685,0.669711
117967,1.26,1.09,248.38,79.500000,35.928320,128.871555,0.561836,0.014110,-0.003782,-0.827120
68050,0.39,0.22,80.48,64.549995,35.928223,128.871224,0.763279,0.001355,0.003681,0.646057
85189,0.83,-0.65,83.99,72.209999,35.928247,128.871300,0.743119,0.009228,0.000607,0.669096


In [34]:
len(shap_values)

5000